<a href="https://colab.research.google.com/github/santhapriyan17/CAS03-CS03/blob/main/CASTILO_Hackathon_team_Imposters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# enhanced model
!pip install scapy seaborn tensorflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import ipaddress
import time
from collections import defaultdict
from scipy.stats import entropy
import random

# Constants
SIMULATION_DURATION = 200  # seconds
PACKET_INTERVAL = 0.001  # seconds
LEARNING_RATE = 0.01
DISCOUNT_FACTOR = 0.99
EPSILON = 0.1
NUM_BOTNET_IPS = 50

class NetworkPacket:
    def __init__(self, src_ip, dst_ip, src_port, dst_port, protocol, size, flags=None):
        self.src_ip = src_ip
        self.dst_ip = dst_ip
        self.src_port = src_port
        self.dst_port = dst_port
        self.protocol = protocol
        self.size = size
        self.flags = flags or {}
        self.timestamp = time.time()

class DDoSDetector:
    def __init__(self):
        self.rf_model = None
        self.if_model = None
        self.autoencoder = None
        self.scaler = StandardScaler()
        self.feature_importance = None
        self.detection_threshold = 0.5
        self.recent_predictions = []
        self.ip_packet_counts = defaultdict(int)
        self.ip_entropy_values = defaultdict(float)
        self.q_table = defaultdict(lambda: np.zeros(3))  # 3 actions: 0 (normal), 1 (potential DDoS), 2 (confirmed DDoS)
        self.botnet_ips = [f"10.0.0.{i}" for i in range(1, NUM_BOTNET_IPS + 1)]
        self.attack_types = [
            self.http2_rapid_reset,
            self.https_flood,
            self.syn_flood,
            self.udp_flood,
            self.dns_amplification,
            self.ntp_amplification,
            self.slowloris,
            self.tcp_sockstress
        ]
        self.mitigation_strategies = {
            "rate_limiting": self.rate_limiting,
            "ip_blocking": self.ip_blocking,
            "traffic_rerouting": self.traffic_rerouting,
            "challenge_response": self.challenge_response
        }
        self.packet_history = []
        self.anomaly_scores = []

    def generate_normal_traffic(self):
        src_ip = f"{np.random.randint(1, 256)}.{np.random.randint(0, 256)}.{np.random.randint(0, 256)}.{np.random.randint(0, 256)}"
        dst_ip = f"{np.random.randint(1, 256)}.{np.random.randint(0, 256)}.{np.random.randint(0, 256)}.{np.random.randint(0, 256)}"
        src_port = np.random.randint(1024, 65536)
        dst_port = np.random.randint(1, 1024)
        protocol = np.random.choice(['TCP', 'UDP', 'ICMP'])
        size = np.random.randint(64, 1500)
        return NetworkPacket(src_ip, dst_ip, src_port, dst_port, protocol, size)

    def http2_rapid_reset(self):
        src_ip = random.choice(self.botnet_ips)
        dst_ip = "192.168.1.1"  # Target server
        src_port = np.random.randint(1024, 65536)
        dst_port = 443
        protocol = 'TCP'
        size = 100  # Small packets
        flags = {'RST': 1, 'HTTP2': True}
        return NetworkPacket(src_ip, dst_ip, src_port, dst_port, protocol, size, flags)

    def https_flood(self):
        src_ip = random.choice(self.botnet_ips)
        dst_ip = "192.168.1.1"  # Target server
        src_port = np.random.randint(1024, 65536)
        dst_port = 443
        protocol = 'TCP'
        size = np.random.randint(1000, 2000)  # Larger packets for HTTPS requests
        flags = {'SSL': True}
        return NetworkPacket(src_ip, dst_ip, src_port, dst_port, protocol, size, flags)

    def syn_flood(self):
        src_ip = random.choice(self.botnet_ips)
        dst_ip = "192.168.1.1"  # Target server
        src_port = np.random.randint(1024, 65536)
        dst_port = 80
        protocol = 'TCP'
        size = 60  # SYN packets are typically small
        flags = {'SYN': 1}
        return NetworkPacket(src_ip, dst_ip, src_port, dst_port, protocol, size, flags)

    def udp_flood(self):
        src_ip = random.choice(self.botnet_ips)
        dst_ip = "192.168.1.1"  # Target server
        src_port = np.random.randint(1024, 65536)
        dst_port = np.random.randint(1, 65536)
        protocol = 'UDP'
        size = np.random.randint(64, 1500)
        return NetworkPacket(src_ip, dst_ip, src_port, dst_port, protocol, size)

    def dns_amplification(self):
        src_ip = "192.168.1.1"  # Spoofed source IP (target)
        dst_ip = f"8.8.{np.random.randint(0, 256)}.{np.random.randint(0, 256)}"  # DNS server
        src_port = 53
        dst_port = np.random.randint(1024, 65536)
        protocol = 'UDP'
        size = np.random.randint(512, 4096)  # DNS responses can be large
        flags = {'DNS': True}
        return NetworkPacket(src_ip, dst_ip, src_port, dst_port, protocol, size, flags)

    def ntp_amplification(self):
        src_ip = "192.168.1.1"  # Spoofed source IP (target)
        dst_ip = f"123.{np.random.randint(0, 256)}.{np.random.randint(0, 256)}.{np.random.randint(0, 256)}"  # NTP server
        src_port = 123
        dst_port = np.random.randint(1024, 65536)
        protocol = 'UDP'
        size = np.random.randint(1000, 5000)  # NTP responses can be very large
        flags = {'NTP': True}
        return NetworkPacket(src_ip, dst_ip, src_port, dst_port, protocol, size, flags)

    def slowloris(self):
        src_ip = random.choice(self.botnet_ips)
        dst_ip = "192.168.1.1"  # Target server
        src_port = np.random.randint(1024, 65536)
        dst_port = 80
        protocol = 'TCP'
        size = 200  # Small packets
        flags = {'ACK': 1, 'Partial': True}
        return NetworkPacket(src_ip, dst_ip, src_port, dst_port, protocol, size, flags)

    def tcp_sockstress(self):
        src_ip = random.choice(self.botnet_ips)
        dst_ip = "192.168.1.1"  # Target server
        src_port = np.random.randint(1024, 65536)
        dst_port = 80
        protocol = 'TCP'
        size = 60  # Small packets
        flags = {'ACK': 1, 'WindowSize': 0}
        return NetworkPacket(src_ip, dst_ip, src_port, dst_port, protocol, size, flags)

    def generate_ddos_traffic(self):
        attack_func = random.choice(self.attack_types)
        return attack_func()

    def generate_mixed_traffic(self, ddos_ratio=0.3):
        if np.random.random() < ddos_ratio:
            return self.generate_ddos_traffic(), 1  # 1 indicates DDoS traffic
        else:
            return self.generate_normal_traffic(), 0  # 0 indicates normal traffic

    def extract_features(self, packet: NetworkPacket):
        features = [
            int(ipaddress.IPv4Address(packet.src_ip)),
            int(ipaddress.IPv4Address(packet.dst_ip)),
            packet.src_port,
            packet.dst_port,
            {'TCP': 0, 'UDP': 1, 'ICMP': 2}.get(packet.protocol, 3),
            packet.size,
            int(packet.flags.get('RST', 0)),
            int(packet.flags.get('HTTP2', 0)),
            int(packet.flags.get('SSL', 0)),
            int(packet.flags.get('SYN', 0)),
            int(packet.flags.get('DNS', 0)),
            int(packet.flags.get('NTP', 0)),
            int(packet.flags.get('Partial', 0)),
            int(packet.flags.get('WindowSize', 1)),
            self.ip_packet_counts[packet.src_ip],
            self.ip_entropy_values[packet.src_ip]
        ]
        return features

    def preprocess_data(self, data):
        X = np.array([self.extract_features(packet) for packet, _ in data])
        y = np.array([label for _, label in data])

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)

        return X_train_scaled, X_test_scaled, y_train, y_test

    def build_autoencoder(self, input_dim):
        input_layer = Input(shape=(input_dim,))
        encoded = Dense(64, activation='relu')(input_layer)
        encoded = Dense(32, activation='relu')(encoded)
        encoded = Dense(16, activation='relu')(encoded)
        decoded = Dense(32, activation='relu')(encoded)
        decoded = Dense(64, activation='relu')(decoded)
        decoded = Dense(input_dim, activation='linear')(decoded)

        autoencoder = Model(input_layer, decoded)
        autoencoder.compile(optimizer='adam', loss='mse')
        return autoencoder

    def train_models(self, X_train, y_train):
        # Random Forest
        self.rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
        self.rf_model.fit(X_train, y_train)
        self.feature_importance = self.rf_model.feature_importances_

        # Isolation Forest
        self.if_model = IsolationForest(contamination=0.1, random_state=42)
        self.if_model.fit(X_train)

        # Autoencoder
        self.autoencoder = self.build_autoencoder(X_train.shape[1])
        self.autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, shuffle=True, validation_split=0.2, verbose=0)

    def evaluate_models(self, X_test, y_test):
        # Random Forest evaluation
        rf_pred = self.rf_model.predict(X_test)
        rf_accuracy = accuracy_score(y_test, rf_pred)
        print(f"\nRandom Forest Accuracy: {rf_accuracy:.4f}")
        print("\nRandom Forest Classification Report:")
        print(classification_report(y_test, rf_pred))

        # Isolation Forest evaluation
        if_pred = self.if_model.predict(X_test)
        if_pred = [1 if x == -1 else 0 for x in if_pred]  # Convert to binary (1 for anomaly, 0 for normal)
        if_accuracy = accuracy_score(y_test, if_pred)
        print(f"\nIsolation Forest Accuracy: {if_accuracy:.4f}")
        print("\nIsolation Forest Classification Report:")
        print(classification_report(y_test, if_pred))

        # Autoencoder evaluation
        reconstructed = self.autoencoder.predict(X_test)
        mse = np.mean(np.power(X_test - reconstructed, 2), axis=1)
        threshold = np.percentile(mse, 95)  # 95th percentile as threshold
        ae_pred = [1 if error > threshold else 0 for error in mse]
        ae_accuracy = accuracy_score(y_test, ae_pred)
        print(f"\nAutoencoder Accuracy: {ae_accuracy:.4f}")
        print("\nAutoencoder Classification Report:")
        print(classification_report(y_test, ae_pred))

        return (rf_accuracy + if_accuracy + ae_accuracy) / 3  # Average accuracy
